# D. PERFORMANCE UNDER NEW WORK CONDITIONS
In this experiment, we will evaluate effect of the proposed few-shot learning method to address the fourth challenge in limited data fault diagnosis. We mainly pay attention in unbalanced working condition. When new categories appear, we would like to evaluate how well the few-shot performance. We use domain adaptation to simulate new work conditions.

In [1]:
%matplotlib inline
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
# set the memory usage
tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True
set_session(tf.Session(config=tf_config))

import matplotlib.pyplot as plt
from scipy.io import loadmat
import numpy as np
import imp
import pandas as pd

import cwru 

import keras
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


## Load models

In [2]:
import models
# imp.reload(models)
window_size = 2048

siamese_net = models.load_siamese_net((window_size,2))
print('\nsiamese_net summary:')
siamese_net.summary()


siamese_net summary:
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 2048, 2)      0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 2048, 2)      0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 100)          51796       input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 100)          0           sequential_1[1][0]    

In [3]:
print('\nsequential_3 is WDCNN:')
siamese_net.layers[2].summary()


sequential_3 is WDCNN:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 128, 16)           2064      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 64, 16)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 64, 32)            1568      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 32, 32)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 32, 64)            4160      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 16, 64)            0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 16, 64)         

In [4]:
wdcnn_net = models.load_wdcnn_net()
print('\nwdcnn_net summary:')
wdcnn_net.summary()

52806

wdcnn_net summary:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 2048, 2)           0         
_________________________________________________________________
sequential_2 (Sequential)    (None, 100)               51796     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                1010      
Total params: 52,806
Trainable params: 52,806
Non-trainable params: 0
_________________________________________________________________


## Config

In [5]:
import keras
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint,EarlyStopping

import siamese
# imp.reload(siamese)
import utils
# imp.reload(utils)

settings = {
  "N_way": 10,           # how many classes for testing one-shot tasks>
  "batch_size": 32,
  "best": -1,
  "evaluate_every": 600,   # interval for evaluating on one-shot tasks
  "loss_every": 20,      # interval for printing loss (iterations)
  "n_iter": 15000,
  "n_val": 2,          #how many one-shot tasks to validate on?
  "n": 0,
  "save_path":"",
  "save_weights_file": "weights-best-10-oneshot-low-data.hdf5"
}
loads ={'A':'1772', 'B':'1750', 'C':'1730'}

times = 10

In [6]:
def WC_train_and_test(exp_name,exps,hps_transfer,is_training):
    window_size = 2048
    for (hp_from,hp_to) in hps_transfer:
        data = cwru.CWRU(['12DriveEndFault'], [loads[i] for i in hp_from],window_size)
        print(len(data.X_train),len(data.X_test))
        data_test = cwru.CWRU(['12DriveEndFault'], [loads[i] for i in hp_to],window_size)
        print(len(data_test.X_train),len(data_test.X_test))
        data.X_test, data.y_test = data_test.X_test, data_test.y_test

        scores_hps = []
        train_classes = sorted(list(set(data.y_train)))
        train_indices = [np.where(data.y_train == i)[0] for i in train_classes]
        for exp in exps:
            # enable the random seed for experimental reproduction
            np.random.seed(exp)
            scores_1_shot = []
            scores_5_shot = []
            scores_wdcnn = []
            num = int(exp/len(train_classes))
            settings['evaluate_every'] = 300 if exp<1000 else 600
            print(settings['evaluate_every'])
            for time_idx in range(times):
                np.random.seed(0)
                print("\n%s-%s"%(exp,time_idx) + '*'*80)
                settings["save_path"] = "tmp/%s/size_%s/%s2%s/time_%s/" % (exp_name,exp,''.join(hp_from),
                                                                           '_'.join(hp_to),time_idx)
                data._mkdir(settings["save_path"])

                train_idxs = []
                val_idxs = []
                for i, c in enumerate(train_classes):
                    select_idx = train_indices[i][np.random.choice(len(train_indices[i]), num, replace=False)]
                    split = int(0.6*num)
                    train_idxs.extend(select_idx[:split])
                    end = num
                    if(0.4*num>100):
                        end = split+100
                    val_idxs.extend(select_idx[split:end])
                X_train, y_train = data.X_train[train_idxs],data.y_train[train_idxs], 
                X_val, y_val = data.X_train[val_idxs],data.y_train[val_idxs], 

                print(train_idxs[0:10])
                print(val_idxs[0:10])
            
                # load one-shot model and training
                siamese_net = models.load_siamese_net()
                siamese_loader = siamese.Siamese_Loader(X_train,
                                                    y_train,
                                                    X_val,
                                                    y_val)
                if(is_training):
                    print(siamese.train_and_test_oneshot(settings,siamese_net,siamese_loader))

                # test 1_shot and 5_shot
                print("load best weights",settings["save_path"] + settings['save_weights_file'])
                siamese_net.load_weights(settings["save_path"] + settings['save_weights_file'])
                siamese_loader.set_val(data.X_test,data.y_test)
                s = 'val'
                preds_5_shot = []
                scores = []
                for k in range(5):
                    val_acc,preds = siamese_loader.test_oneshot2(siamese_net,len(siamese_loader.classes[s]),
                                                                 len(siamese_loader.data[s]),verbose=False)
        #             confusion_plot(preds[:,1],preds[:,0])
                    print(val_acc)
                    scores.append(val_acc)
                    preds_5_shot.append(preds[:,1])
                preds = []
                for line in np.array(preds_5_shot).T:
                    preds.append(np.argmax(np.bincount(line)))
        #         confusion_plot(np.array(preds),data.y_test) 
                score_5_shot = accuracy_score(data.y_test,np.array(preds))*100
                print('5_shot:',score_5_shot)
                scores_1_shot.append(scores[0])
                scores_5_shot.append(score_5_shot)


                # load wdcnn model and training
                y_train = keras.utils.to_categorical(y_train, data.nclasses)
                y_val = keras.utils.to_categorical(y_val, data.nclasses)
                y_test = keras.utils.to_categorical(data.y_test, data.nclasses)

                earlyStopping = EarlyStopping(monitor='val_loss', patience=20, verbose=0, mode='min')
                # checkpoint
                filepath="%s/weights-best-10-cnn-low-data.hdf5" % (settings["save_path"])
                checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')
                tb = keras.callbacks.TensorBoard(log_dir='./tmp/logs', histogram_freq=4, 
                                                 batch_size=32, write_graph=True, 
                                                 write_grads=False, write_images=True, 
                                                 embeddings_freq=0, embeddings_layer_names=None, 
                                                 embeddings_metadata=None, embeddings_data=None, 
                                                 update_freq='epoch')

                # callbacks_list = [earlyStopping,checkpoint,tb]
                callbacks_list = [earlyStopping,checkpoint]

                wdcnn_net = models.load_wdcnn_net()

                if(is_training):
                    wdcnn_net.fit(X_train, y_train,
                              batch_size=32,
                              epochs=300,
                              verbose=0,
                              callbacks=callbacks_list,
                              validation_data=(X_val, y_val))

                # test wdcnn
                wdcnn_net.load_weights(filepath)
                score = wdcnn_net.evaluate(data.X_test, y_test, verbose=0)
                print('Test loss:', score[0])
                print('Test accuracy:', score[1])
                scores_wdcnn.append(score[1]*100)


            a =pd.DataFrame(np.array(scores_1_shot))
    #         a.columns = str(exp)
            a.to_csv("tmp/%s/size_%s/%s2%s/scores_1_shot.csv" % (exp_name,exp,''.join(hp_from),'_'.join(hp_to)),index=True)


            a =pd.DataFrame(np.array(scores_5_shot))
    #         a.columns = str(exp)
            a.to_csv("tmp/%s/size_%s/%s2%s/scores_5_shot.csv" % (exp_name,exp,''.join(hp_from),'_'.join(hp_to)),index=True)


            a =pd.DataFrame(np.array(scores_wdcnn))
    #         a.columns = str(exp)
            a.to_csv("tmp/%s/size_%s/%s2%s/scores_wdcnn.csv" % (exp_name,exp,''.join(hp_from),'_'.join(hp_to)),index=True)  

        
def WC_analysis(exp_name,exps,hps_transfer):
    scores_1_shot_all = pd.DataFrame()
    scores_5_shot_all = pd.DataFrame()
    scores_wdcnn_all = pd.DataFrame()
    for (hp_from,hp_to) in hps_transfer:
        for exp in exps:
            hps = "%s2%s" % (''.join(hp_from),'_'.join(hp_to))
            file_path = "tmp/%s/size_%s/%s" % (exp_name,exp,hps)
            tmp_data = pd.read_csv("%s/scores_1_shot.csv" % (file_path), 
                                   sep=',', index_col=0)
            tmp_data['hps'] = hps
            tmp_data['exp'] = exp 
            scores_1_shot_all = pd.concat([scores_1_shot_all,tmp_data],axis=0)

            tmp_data = pd.read_csv("%s/scores_5_shot.csv" % (file_path), 
                                   sep=',', index_col=0)
            tmp_data['hps'] = hps
            tmp_data['exp'] = exp 
            scores_5_shot_all = pd.concat([scores_5_shot_all,tmp_data],axis=0)

            tmp_data = pd.read_csv("%s/scores_wdcnn.csv" % (file_path), 
                                   sep=',', index_col=0)
            tmp_data['hps'] = hps
            tmp_data['exp'] = exp 
            scores_wdcnn_all = pd.concat([scores_wdcnn_all,tmp_data],axis=0)


    scores_1_shot_all.to_csv("tmp/%s/scores_1_shot_all.csv" % (exp_name), float_format='%.6f', index=True)
    scores_5_shot_all.to_csv("tmp/%s/scores_5_shot_all.csv" % (exp_name), float_format='%.6f', index=True)
    scores_wdcnn_all.to_csv("tmp/%s/scores_wdcnn_all.csv" % (exp_name), float_format='%.6f', index=True)

    scores_1_shot_all['model'] = 'One-shot'
    scores_5_shot_all['model'] = 'Five-shot'
    scores_wdcnn_all['model'] = 'WDCNN'

    scores_all = pd.concat([scores_1_shot_all,scores_5_shot_all,scores_wdcnn_all],axis=0)
    scores_all.to_csv("tmp/%s/scores_all.csv" % (exp_name), float_format='%.6f', index=True)   
    
    return scores_all

## Scenario setting for domain adaption

In [7]:
exp_name = "EXP-D"
exps = [90,6600]
hps_transfer = [
    (['A'],['B']),
    (['A'],['C']),
    (['B'],['A']),
    (['B'],['C']),
    (['C'],['A']),
    (['C'],['B']),
]

is_training = False    # enable or disable train models. if enable training, save best models will be update.

### Training & Testing

In [ ]:
WC_train_and_test(exp_name,exps,hps_transfer,is_training)

### Analysis

In [8]:
scores_all = WC_analysis(exp_name,exps,hps_transfer)
scores_all_mean = scores_all.groupby(['exp','model','hps']).mean().unstack()\
                .sort_values(by=['model'],ascending=0).sort_values(by=['exp'],ascending=1)
scores_all_std = scores_all.groupby(['exp','model','hps']).std().unstack()\
                .sort_values(by=['model'],ascending=0).sort_values(by=['exp'],ascending=1)
scores_all_mean.to_csv("tmp/%s/scores_all_mean.csv" % (exp_name), float_format='%.2f', index=True)
scores_all_std.to_csv("tmp/%s/scores_all_std.csv" % (exp_name), float_format='%.2f', index=True)
scores_all_mean, scores_all_std

(                    0                                   
 hps               A2B    A2C    B2A    B2C    C2A    C2B
 exp  model                                              
 90   WDCNN      76.96  77.92  75.00  78.24  75.00  80.68
      One-shot   84.24  79.80  79.24  77.28  71.16  74.00
      Five-shot  86.60  82.08  80.84  79.04  71.96  74.96
 6600 WDCNN      97.08  91.48  93.00  91.80  78.84  85.88
      One-shot   98.92  90.44  85.36  90.12  60.36  65.36
      Five-shot  99.24  90.40  85.28  90.32  60.52  65.36,
                        0                                                    
 hps                  A2B        A2C       B2A        B2C       C2A       C2B
 exp  model                                                                  
 90   WDCNN      8.099822   8.021748  9.618963   6.767767  5.936703  5.376657
      One-shot   5.287133   6.069962  5.186135  10.377411  4.951139  8.504117
      Five-shot  5.443038   5.926363  4.531176  10.842222  4.827514  9.373627
 6600 WDC

## Scenario setting for A2C and C2A

In [9]:
exp_name = "EXP-D"
# 90 and 6600 exp have trained done before
exps = [120,300,900,1500,3000]
hps_transfer = [
#     (['A'],['B']),
    (['A'],['C']),
#     (['B'],['A']),
#     (['B'],['C']),
    (['C'],['A']),
#     (['C'],['B']),
]

### Training & Testing

In [ ]:
WC_train_and_test(exp_name,exps,hps_transfer,is_training)

### Analysis

In [10]:
# analysis
exps = [90,120,300,900,1500,3000,6600]
scores_all = WC_analysis(exp_name,exps,hps_transfer)
scores_all_mean = scores_all.groupby(['exp','model','hps']).mean().unstack().unstack().T \
            .sort_values(by=['model'],ascending=0).sort_values(by=['hps'],ascending=1)
scores_all_std = scores_all.groupby(['exp','model','hps']).std().unstack().unstack().T \
            .sort_values(by=['model'],ascending=0).sort_values(by=['hps'],ascending=1)
scores_all_mean.to_csv("tmp/%s/scores_AC_mean.csv" % (exp_name), float_format='%.2f', index=True)
scores_all_std.to_csv("tmp/%s/scores_AC_std.csv" % (exp_name), float_format='%.2f', index=True)
scores_all_mean, scores_all_std

(exp               90     120    300    900    1500   3000   6600
   hps model                                                     
 0 A2C WDCNN      77.92  81.00  94.12  93.52  96.44  94.04  91.48
       One-shot   79.80  86.84  85.80  91.60  92.36  90.96  90.44
       Five-shot  82.08  88.48  86.92  92.52  92.40  91.00  90.40
   C2A WDCNN      75.00  73.52  78.00  73.72  78.92  75.16  78.84
       One-shot   71.16  70.40  67.12  63.68  62.00  60.64  60.36
       Five-shot  71.96  70.64  67.20  63.68  62.28  60.56  60.52,
 exp                  90        120       300       900       1500      3000  \
   hps model                                                                   
 0 A2C WDCNN      8.021748  7.669275  2.945731  3.727615  1.773384  5.601428   
       One-shot   6.069962  5.502161  7.962691  3.567757  3.509733  6.972199   
       Five-shot  5.926363  5.935355  8.394813  3.155524  3.868965  7.352097   
   C2A WDCNN      5.936703  4.360632  5.222175  4.891898  5.891010  4.8

## New scenario setting for domain adaption

In [11]:
exp_name = "EXP-D-NEW"
exps = [90,13200]
hps_transfer = [
    (['A','B'],['C']),
    (['A','C'],['B']),
    (['B','C'],['A']),
]

### Training & Testing

In [ ]:
WC_train_and_test(exp_name,exps,hps_transfer,is_training)

### Analysis

In [12]:
scores_all = WC_analysis(exp_name,exps,hps_transfer)
scores_all_mean = scores_all.groupby(['exp','model','hps']).mean().unstack()\
                .sort_values(by=['model'],ascending=0).sort_values(by=['exp'],ascending=1)
scores_all_std = scores_all.groupby(['exp','model','hps']).std().unstack()\
                .sort_values(by=['model'],ascending=0).sort_values(by=['exp'],ascending=1)
scores_all_mean.to_csv("tmp/%s/scores_all_mean.csv" % (exp_name), float_format='%.2f', index=True)
scores_all_std.to_csv("tmp/%s/scores_all_std.csv" % (exp_name), float_format='%.2f', index=True)
scores_all_mean, scores_all_std

(                     0              
 hps               AB2C   AC2B   BC2A
 exp   model                         
 90    WDCNN      73.32  80.72  75.24
       One-shot   83.72  87.76  81.24
       Five-shot  85.12  90.56  83.24
 13200 WDCNN      96.04  99.28  94.04
       One-shot   96.84  99.80  93.96
       Five-shot  97.00  99.84  94.32,
                         0                    
 hps                  AB2C      AC2B      BC2A
 exp   model                                  
 90    WDCNN      9.339141  8.112103  6.771049
       One-shot   7.339513  4.561481  3.673085
       Five-shot  6.326804  3.642710  4.154569
 13200 WDCNN      4.507574  1.395867  3.479208
       One-shot   3.970782  0.388730  5.148290
       Five-shot  3.906689  0.386437  4.743370)